In [1]:
import sys 

In [2]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

/coc/pskynet3/hagrawal9/project/habitat/habitat-api


In [23]:
import gzip
import json
import os
import sys
from typing import Any, Dict, List, Optional, Type

import attr
import cv2
import git
import magnum as mn
import numpy as np

# %matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


import habitat
import habitat_sim
from habitat.config import Config
from habitat.core.registry import registry
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations import maps, fog_of_war
from habitat.utils.visualizations.utils import observations_to_image

from rearrangement.utils.planner import (
    compute_traversable_map,
    compute_distance_using_fmm,
    find_dist_from_map,
    compute_distance_mat_using_navmesh,
    compute_distance_mat_using_fmm,
    find_shortest_path_for_multiple_objects
)
from rearrangement.utils.visualization import (
    get_top_down_map
)
from rearrangement.utils.geometry import (
     geodesic_distance
)
from collections import defaultdict

In [24]:
from habitat.datasets.rearrangement.rearrangement_dataset import RearrangementDatasetV0
from habitat.tasks.rearrangement.rearrangement_task import RearrangementEpisode

In [25]:
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
# %cd $dir_path
data_path = os.path.join(dir_path, "data")
output_directory = "data/tutorials/output/"  # @param {type:"string"}
output_path = os.path.join(dir_path, output_directory)

In [26]:
config = habitat.get_config("configs/tasks/rearrangement_gibson.yaml")

In [27]:
config.defrost()
config.TASK.MEASUREMENTS = ['OBJECT_TO_GOAL_DISTANCE', 'AGENT_TO_OBJECT_DISTANCE', 'TOP_DOWN_MAP']
config.freeze()

In [28]:
config.TASK.TOP_DOWN_MAP.MAP_RESOLUTION

256

In [29]:
try:
    env.close()
except:
    pass
env = habitat.Env(config)
fog_of_war_mask = None
pickup_order = {}


2020-10-13 14:48:12,015 Initializing dataset RearrangementDataset-v0
2020-10-13 14:48:15,797 initializing sim RearrangementSim-v0
2020-10-13 14:48:20,171 Initializing task RearrangementTask-v0


In [30]:
# # Scratch Code
# for objid in env._sim.get_existing_object_ids():
#     if objid != env._sim.agent_object_id:
#         episode_obj_id = env._task.sim_object_to_objid_mapping[objid]
#         dist = geodesic_distance(env._sim.pathfinder, agent_pos, env.current_episode.objects[episode_obj_id].position)
#         if dist == np.inf:
#             print("Voila!!")


In [31]:
def compute_oracle_pickup_order(env):
    obs = env.reset()
    metrics = env.get_metrics()
    episode = env.current_episode
    metrics = env.get_metrics()

    agent_pos = env._sim.get_agent(0).get_state().position
    object_positions = [obj.position for obj in episode.objects]
    goal_positions = [obj.position for obj in episode.goals]
    
    dist_mat = compute_distance_mat_using_navmesh(env._task._simple_pathfinder, agent_pos, object_positions, goal_positions)
    route_indexes_navmesh, pickup_order_navmesh = find_shortest_path_for_multiple_objects(dist_mat)
    
#     top_down_map, fog_of_war_mask = get_top_down_map(
#         env, env._task._simple_pathfinder, ignore_objects=True, fog_of_war_mask=None, draw_fow=False, 
#         draw_agent=False, draw_object_start_pos=False, draw_object_final_pos=False, draw_object_curr_pos=False
#     )
    
    
#     a_y, a_x = maps.to_grid(
#         agent_position[2],
#         agent_position[0],
#         top_down_map.shape[0:2],
#         sim=env._sim,
#     )
#     grid_object_positions = []
#     grid_goal_positions = []

#     for i, obj_pos in enumerate(object_positions):
#         tdm_pos = maps.to_grid(
#             obj_pos[2],
#             obj_pos[0],
#             top_down_map.shape[0:2],
#             sim=env._sim,
#         )
#         grid_object_positions.append(tdm_pos)

#     for i, goal_pos in enumerate(goal_positions):
#         tdm_pos = maps.to_grid(
#             goal_pos[2],
#             goal_pos[0],
#             top_down_map.shape[0:2],
#             sim=env._sim,
#         )

#         grid_goal_positions.append(tdm_pos)

#     tdmap = np.copy(top_down_map[:, :, 0])
#     tdmap = tdmap / np.max(tdmap)
#     dist_mat_map = compute_distance_mat_using_fmm(tdmap, [a_y, a_x], grid_object_positions, grid_goal_positions)
    
    
#     route_indexes_map, pickup_order_map = find_shortest_path_for_multiple_objects(dist_mat_map/10)
    
    return {
        'episode_id': episode.episode_id, 
        'scene_id': episode.scene_id,
        'pickup_order': pickup_order_navmesh,
#         'pickup_order_fmm': pickup_order_map 
    }

In [32]:
while(len(pickup_order)!=35000):
    data = compute_oracle_pickup_order(env)
    pickup_order[data['episode_id'] + '_' + data['scene_id']] = data
    print('\rEpisode ID: {} Done: {}'.format(data['episode_id'], len(pickup_order)), end=" ")

Episode ID: 939 Done: 35000 

In [34]:
with open('data/pickup_order.json','w') as f:
    json.dump(pickup_order, f)

In [ ]:
scene_episodes = defaultdict(list)
for episode in env.episodes:
    scene_episodes[episode.scene_id].append(episode.episode_id)

In [21]:
for key, value in scene_episodes.items():
    print(len(value), key)

1000 data/scene_datasets/gibson_train_val/Montreal.glb
